In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Embedding, LSTM, Dense, Dropout,
    Layer, MultiHeadAttention, LayerNormalization,
    Concatenate, GlobalAveragePooling1D
)
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

class UserProductInteractionLayer(Layer):
    """
    Custom layer for capturing user-product interaction patterns
    """
    def __init__(self, units, num_heads=4, **kwargs):
        super(UserProductInteractionLayer, self).__init__(**kwargs)
        self.units = units
        self.num_heads = num_heads

    def build(self, input_shape):
        # Implement multi-head attention mechanism
        self.multi_head_attention = MultiHeadAttention(
            num_heads=self.num_heads,
            key_dim=self.units
        )
        self.layer_norm = LayerNormalization()
        super(UserProductInteractionLayer, self).build(input_shape)

    def call(self, inputs):
        # Assume inputs are [text_embedding, user_embedding, product_embedding]
        text_embed, user_embed, product_embed = inputs

        # Interactive attention
        attention_output = self.multi_head_attention(
            query=text_embed,
            value=user_embed,
            key=product_embed
        )

        # Residual connection and layer normalization
        interaction_output = self.layer_norm(text_embed + attention_output)

        return interaction_output

class MIANAReviewPredictor:
    def __init__(self, max_words=5000, max_len=100):
        # Create necessary directories
        self.base_dir = 'weights'
        os.makedirs(self.base_dir, exist_ok=True)
        os.makedirs('metrics', exist_ok=True)

        self.max_words = max_words
        self.max_len = max_len
        self.tokenizer = Tokenizer(num_words=max_words)
        self.label_encoder = LabelEncoder()

    def preprocess_data(self, df):
        # Tokenize text
        self.tokenizer.fit_on_texts(df['text_'])
        text_sequences = self.tokenizer.texts_to_sequences(df['text_'])
        text_padded = pad_sequences(text_sequences, maxlen=self.max_len)

        # Encode labels
        labels = self.label_encoder.fit_transform(df['label'])

        return text_padded, labels

    def create_miana_model(self, vocab_size, num_classes):
        # Text input
        text_input = Input(shape=(self.max_len,), name='text_input')

        # Embedding layer
        embedding = Embedding(
            vocab_size,
            256,
            input_length=self.max_len
        )(text_input)

        # Word-Level Fusion Module (LSTM-based)
        lstm1 = LSTM(256, return_sequences=True, name='word_level_lstm1')(embedding)
        lstm2 = LSTM(128, return_sequences=True, name='word_level_lstm2')(lstm1)

        # User-Product Interaction Layer
        interaction_layer = UserProductInteractionLayer(
            units=128,
            num_heads=4
        )([lstm2, lstm2, lstm2])  # Using same embedding for demonstration

        # Sentence-Level Interactive Attention
        multi_head_attention = MultiHeadAttention(
            num_heads=4,
            key_dim=128
        )(query=interaction_layer, value=interaction_layer)

        # Pooling and Dropout
        pooled = GlobalAveragePooling1D()(multi_head_attention)
        dropout = Dropout(0.5)(pooled)

        # Output layer
        output = Dense(num_classes, activation='softmax', name='output')(dropout)

        # Compile the model
        model = Model(inputs=text_input, outputs=output)
        model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        return model

    def train_and_save(self, df, test_size=0.2, epochs=5, batch_size=32):
        try:
            # Preprocess data
            X, y = self.preprocess_data(df)

            # Split data
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=test_size, random_state=42
            )

            # Create MIANA model
            vocab_size = len(self.tokenizer.word_index) + 1
            num_classes = len(np.unique(y))

            model = self.create_miana_model(vocab_size, num_classes)

            # Training
            history = model.fit(
                X_train, y_train,
                validation_data=(X_test, y_test),
                epochs=epochs,
                batch_size=batch_size,
                verbose=1
            )

            # Ensure correct weight file naming
            weights_path = os.path.join(self.base_dir, 'miana_model.weights.h5')
            model.save_weights(weights_path)

            # Save additional artifacts
            import joblib
            import pickle

            # Save label encoder
            label_encoder_path = os.path.join(self.base_dir, 'label_encoder.joblib')
            joblib.dump(self.label_encoder, label_encoder_path)

            # Save tokenizer
            tokenizer_path = os.path.join(self.base_dir, 'tokenizer.pickle')
            with open(tokenizer_path, 'wb') as handle:
                pickle.dump(self.tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

            # Generate metrics visualization
            self._generate_metrics_visualization(history)

            print(f"Model weights saved to: {weights_path}")
            print(f"Label encoder saved to: {label_encoder_path}")
            print(f"Tokenizer saved to: {tokenizer_path}")

            return model, history

        except Exception as e:
            print(f"Training error: {e}")
            import traceback
            traceback.print_exc()
            return None, None

    def _generate_metrics_visualization(self, history):
        import matplotlib.pyplot as plt

        plt.figure(figsize=(12, 5))

        # Loss subplot
        plt.subplot(1, 2, 1)
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('MIANA Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()

        # Accuracy subplot
        plt.subplot(1, 2, 2)
        plt.plot(history.history['accuracy'], label='Training Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title('MIANA Model Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()

        plt.tight_layout()
        plt.savefig('metrics/miana_training_metrics.png')
        plt.close()

# Main execution
if __name__ == "__main__":
    # Suppress TensorFlow warnings
    import os
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

    # Load reviews dataset
    df = pd.read_csv('/content/reviews.csv')  # Replace with your actual dataset

    # Initialize MIANA predictor
    miana_predictor = MIANAReviewPredictor()

    # Train and save the MIANA model
    miana_predictor.train_and_save(df)

    print("MIANA Model training completed.")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 38s 33ms/step - accuracy: 0.8243 - loss: 0.3708 - val_accuracy: 0.9236 - val_loss: 0.2035
Epoch 2/5
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 33s 33ms/step - accuracy: 0.9475 - loss: 0.1442 - val_accuracy: 0.9442 - val_loss: 0.1538
Epoch 3/5
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 41s 32ms/step - accuracy: 0.9671 - loss: 0.0912 - val_accuracy: 0.9369 - val_loss: 0.1618
Epoch 4/5
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 41s 32ms/step - accuracy: 0.9770 - loss: 0.0642 - val_accuracy: 0.9405 - val_loss: 0.1618
Epoch 5/5
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 41s 32ms/step - accuracy: 0.9851 - loss: 0.0444 - val_accuracy: 0.9393 - val_loss: 0.2600
Model weights saved to: weights/miana_model.weights.h5
Label encoder saved to: weights/label_encoder.joblib
Tokenizer saved to: weights/tokenizer.pickle
MIANA Model training completed.


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Embedding, LSTM, Dense, Dropout,
    Layer, MultiHeadAttention, LayerNormalization,
    Concatenate, GlobalAveragePooling1D
)
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    precision_recall_curve,
    roc_curve,
    auc
)
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import pickle

class UserProductInteractionLayer(Layer):
    """
    Custom layer for capturing user-product interaction patterns
    """
    def __init__(self, units, num_heads=4, **kwargs):
        super(UserProductInteractionLayer, self).__init__(**kwargs)
        self.units = units
        self.num_heads = num_heads

    def build(self, input_shape):
        # Implement multi-head attention mechanism
        self.multi_head_attention = MultiHeadAttention(
            num_heads=self.num_heads,
            key_dim=self.units
        )
        self.layer_norm = LayerNormalization()
        super(UserProductInteractionLayer, self).build(input_shape)

    def call(self, inputs):
        # Assume inputs are [text_embedding, user_embedding, product_embedding]
        text_embed, user_embed, product_embed = inputs

        # Interactive attention
        attention_output = self.multi_head_attention(
            query=text_embed,
            value=user_embed,
            key=product_embed
        )

        # Residual connection and layer normalization
        interaction_output = self.layer_norm(text_embed + attention_output)

        return interaction_output

class MIANAReviewPredictor:
    def __init__(self, max_words=5000, max_len=100):
        # Create necessary directories
        self.base_dir = 'weights'
        os.makedirs(self.base_dir, exist_ok=True)
        os.makedirs('metrics', exist_ok=True)

        self.max_words = max_words
        self.max_len = max_len
        self.tokenizer = Tokenizer(num_words=max_words)
        self.label_encoder = LabelEncoder()

    def preprocess_data(self, df):
        # Tokenize text
        self.tokenizer.fit_on_texts(df['text_'])
        text_sequences = self.tokenizer.texts_to_sequences(df['text_'])
        text_padded = pad_sequences(text_sequences, maxlen=self.max_len)

        # Encode labels
        labels = self.label_encoder.fit_transform(df['label'])

        return text_padded, labels

    def create_miana_model(self, vocab_size, num_classes):
        # Text input
        text_input = Input(shape=(self.max_len,), name='text_input')

        # Embedding layer
        embedding = Embedding(
            vocab_size,
            256,
            input_length=self.max_len
        )(text_input)

        # Word-Level Fusion Module (LSTM-based)
        lstm1 = LSTM(256, return_sequences=True, name='word_level_lstm1')(embedding)
        lstm2 = LSTM(128, return_sequences=True, name='word_level_lstm2')(lstm1)

        # User-Product Interaction Layer
        interaction_layer = UserProductInteractionLayer(
            units=128,
            num_heads=4
        )([lstm2, lstm2, lstm2])  # Using same embedding for demonstration

        # Sentence-Level Interactive Attention
        multi_head_attention = MultiHeadAttention(
            num_heads=4,
            key_dim=128
        )(query=interaction_layer, value=interaction_layer)

        # Pooling and Dropout
        pooled = GlobalAveragePooling1D()(multi_head_attention)
        dropout = Dropout(0.5)(pooled)

        # Output layer
        output = Dense(num_classes, activation='softmax', name='output')(dropout)

        # Compile the model
        model = Model(inputs=text_input, outputs=output)
        model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        return model

    def train_and_save(self, df, test_size=0.2, epochs=5, batch_size=32):
        try:
            # Preprocess data
            X, y = self.preprocess_data(df)

            # Split data
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=test_size, random_state=42
            )

            # Create MIANA model
            vocab_size = len(self.tokenizer.word_index) + 1
            num_classes = len(np.unique(y))

            model = self.create_miana_model(vocab_size, num_classes)

            # Training
            history = model.fit(
                X_train, y_train,
                validation_data=(X_test, y_test),
                epochs=epochs,
                batch_size=batch_size,
                verbose=1
            )

            # Ensure correct weight file naming
            weights_path = os.path.join(self.base_dir, 'miana_model.weights.h5')
            model.save_weights(weights_path)

            # Save additional artifacts
            # Save label encoder
            label_encoder_path = os.path.join(self.base_dir, 'label_encoder.joblib')
            joblib.dump(self.label_encoder, label_encoder_path)

            # Save tokenizer
            tokenizer_path = os.path.join(self.base_dir, 'tokenizer.pickle')
            with open(tokenizer_path, 'wb') as handle:
                pickle.dump(self.tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

            # Generate comprehensive metrics visualization
            self._generate_comprehensive_metrics(model, X_test, y_test, history)

            print(f"Model weights saved to: {weights_path}")
            print(f"Label encoder saved to: {label_encoder_path}")
            print(f"Tokenizer saved to: {tokenizer_path}")

            return model, history

        except Exception as e:
            print(f"Training error: {e}")
            import traceback
            traceback.print_exc()
            return None, None

    def _generate_comprehensive_metrics(self, model, X_test, y_test, history):
        import matplotlib.pyplot as plt
        import seaborn as sns

        # Predict on test data
        y_pred = model.predict(X_test)
        y_pred_classes = np.argmax(y_pred, axis=1)

        # 1. Training History Metrics
        plt.figure(figsize=(15, 10))
        plt.suptitle('MIANA Model Training Analysis', fontsize=16)

        # Loss subplot
        plt.subplot(2, 2, 1)
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('Training and Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()

        # Accuracy subplot
        plt.subplot(2, 2, 2)
        plt.plot(history.history['accuracy'], label='Training Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title('Training and Validation Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()

        # 2. Confusion Matrix
        plt.subplot(2, 2, 3)
        cm = confusion_matrix(y_test, y_pred_classes)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title('Confusion Matrix')
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')

        # 3. Classification Report
        plt.subplot(2, 2, 4)
        class_report = classification_report(
            y_test,
            y_pred_classes,
            target_names=self.label_encoder.classes_,
            output_dict=True
        )
        report_data = []
        for key, row in class_report.items():
            if isinstance(row, dict):
                report_data.append([
                    key,
                    round(row['precision'], 2),
                    round(row['recall'], 2),
                    round(row['f1-score'], 2)
                ])

        plt.axis('off')
        plt.table(
            cellText=report_data,
            colLabels=['Class', 'Precision', 'Recall', 'F1-Score'],
            loc='center',
            cellLoc='center'
        )
        plt.title('Classification Report')

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.savefig('metrics/miana_comprehensive_metrics.png')
        plt.close()

        # 4. ROC Curves and Precision-Recall Curves
        plt.figure(figsize=(15, 5))

        # ROC Curve
        plt.subplot(1, 2, 1)
        # One-vs-Rest ROC
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(len(self.label_encoder.classes_)):
            fpr[i], tpr[i], _ = roc_curve(
                (y_test == i).ravel(),
                y_pred[:, i].ravel()
            )
            roc_auc[i] = auc(fpr[i], tpr[i])
            plt.plot(
                fpr[i], tpr[i],
                label=f'ROC curve (class {self.label_encoder.classes_[i]}, area = {roc_auc[i]:.2f})'
            )
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc="lower right")

        # Precision-Recall Curve
        plt.subplot(1, 2, 2)
        for i in range(len(self.label_encoder.classes_)):
            precision, recall, _ = precision_recall_curve(
                (y_test == i).ravel(),
                y_pred[:, i].ravel()
            )
            plt.plot(
                recall, precision,
                label=f'Precision-Recall curve (class {self.label_encoder.classes_[i]})'
            )
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision-Recall Curve')
        plt.legend(loc="lower left")

        plt.tight_layout()
        plt.savefig('metrics/miana_roc_pr_curves.png')
        plt.close()

# Main execution
if __name__ == "__main__":
    # Suppress TensorFlow warnings
    import os
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

    # Load reviews dataset
    df = pd.read_csv('/content/reviews.csv')  # Replace with your actual dataset

    # Initialize MIANA predictor
    miana_predictor = MIANAReviewPredictor()

    # Train and save the MIANA model
    miana_predictor.train_and_save(df)

    print("MIANA Model training completed.")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 88s 35ms/step - accuracy: 0.8297 - loss: 0.3640 - val_accuracy: 0.9258 - val_loss: 0.1839
Epoch 2/5
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 34s 33ms/step - accuracy: 0.9443 - loss: 0.1483 - val_accuracy: 0.9389 - val_loss: 0.1687
Epoch 3/5
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 41s 34ms/step - accuracy: 0.9666 - loss: 0.0921 - val_accuracy: 0.9437 - val_loss: 0.1558
Epoch 4/5
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 41s 34ms/step - accuracy: 0.9748 - loss: 0.0711 - val_accuracy: 0.9427 - val_loss: 0.1835
Epoch 5/5
1011/1011 ━━━━━━━━━━━━━━━━━━━━ 41s 34ms/step - accuracy: 0.9820 - loss: 0.0483 - val_accuracy: 0.9402 - val_loss: 0.2159
253/253 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step
Model weights saved to: weights/miana_model.weights.h5
Label encoder saved to: weights/label_encoder.joblib
Tokenizer saved to: weights/tokenizer.pickle
MIANA Model training completed.


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import joblib
import pickle

class MIANAReviewInference:
    def __init__(self, base_dir='weights'):
        """
        Initialize the inference model with pre-trained weights and preprocessing artifacts

        Args:
            base_dir (str): Directory containing saved model artifacts
        """
        self.base_dir = base_dir

        # Load tokenizer
        tokenizer_path = os.path.join(base_dir, 'tokenizer.pickle')
        with open(tokenizer_path, 'rb') as handle:
            self.tokenizer = pickle.load(handle)

        # Load label encoder
        label_encoder_path = os.path.join(base_dir, 'label_encoder.joblib')
        self.label_encoder = joblib.load(label_encoder_path)

        # Model configuration parameters
        self.max_len = 100  # Must match training configuration
        self.max_words = 5000  # Must match training configuration

        # Create and load model
        self.model = self._load_model()

    def _load_model(self):
        """
        Recreate and load pre-trained model weights

        Returns:
            Loaded Keras model with pre-trained weights
        """
        from tensorflow.keras.layers import (
            Input, Embedding, LSTM, Dense, Dropout,
            Layer, MultiHeadAttention, LayerNormalization,
            GlobalAveragePooling1D
        )
        from tensorflow.keras.models import Model

        class UserProductInteractionLayer(Layer):
            def __init__(self, units, num_heads=4, **kwargs):
                super(UserProductInteractionLayer, self).__init__(**kwargs)
                self.units = units
                self.num_heads = num_heads

            def build(self, input_shape):
                self.multi_head_attention = MultiHeadAttention(
                    num_heads=self.num_heads,
                    key_dim=self.units
                )
                self.layer_norm = LayerNormalization()
                super(UserProductInteractionLayer, self).build(input_shape)

            def call(self, inputs):
                text_embed, user_embed, product_embed = inputs
                attention_output = self.multi_head_attention(
                    query=text_embed,
                    value=user_embed,
                    key=product_embed
                )
                interaction_output = self.layer_norm(text_embed + attention_output)
                return interaction_output

        # Recreate model architecture
        text_input = Input(shape=(self.max_len,), name='text_input')

        embedding = Embedding(
            len(self.tokenizer.word_index) + 1,
            256,
            input_length=self.max_len
        )(text_input)

        lstm1 = LSTM(256, return_sequences=True, name='word_level_lstm1')(embedding)
        lstm2 = LSTM(128, return_sequences=True, name='word_level_lstm2')(lstm1)

        interaction_layer = UserProductInteractionLayer(
            units=128,
            num_heads=4
        )([lstm2, lstm2, lstm2])

        multi_head_attention = MultiHeadAttention(
            num_heads=4,
            key_dim=128
        )(query=interaction_layer, value=interaction_layer)

        pooled = GlobalAveragePooling1D()(multi_head_attention)
        dropout = Dropout(0.5)(pooled)

        output = Dense(len(self.label_encoder.classes_), activation='softmax', name='output')(dropout)

        model = Model(inputs=text_input, outputs=output)
        model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        # Load pre-trained weights
        weights_path = os.path.join(self.base_dir, 'miana_model.weights.h5')
        model.load_weights(weights_path)

        return model

    def preprocess_text(self, text):
        """
        Preprocess input text for model prediction

        Args:
            text (str): Input review text

        Returns:
            Preprocessed and padded text sequence
        """
        # Tokenize and pad the text
        text_sequence = self.tokenizer.texts_to_sequences([text])
        text_padded = pad_sequences(
            text_sequence,
            maxlen=self.max_len,
            padding='post',
            truncating='post'
        )

        return text_padded

    def predict(self, review_text, top_k=3):
        """
        Predict label for the given review text

        Args:
            review_text (str): Input review text
            top_k (int): Number of top predictions to return

        Returns:
            List of top predictions with probabilities
        """
        # Preprocess text
        processed_text = self.preprocess_text(review_text)

        # Predict probabilities
        prediction = self.model.predict(processed_text)[0]

        # Get top-k predictions
        top_indices = prediction.argsort()[-top_k:][::-1]
        top_predictions = [
            {
                'label': self.label_encoder.classes_[idx],
                'probability': float(prediction[idx])
            }
            for idx in top_indices
        ]

        return top_predictions

def main():
    # Example usage
    inference_model = MIANAReviewInference()

    # Example reviews
    reviews = [
        "This product is amazing and works perfectly!",
        "I'm very disappointed with the quality of this item.",
        "Decent product, but could use some improvements."
    ]

    # Predict for each review
    for review in reviews:
        predictions = inference_model.predict(review)
        print(f"\nReview: {review}")
        print("Top Predictions:")
        for pred in predictions:
            print(f"- {pred['label']}: {pred['probability']:.2%}")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 56 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 790ms/step

Review: This product is amazing and works perfectly!
Top Predictions:
- OR: 96.11%
- CG: 3.89%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step

Review: I'm very disappointed with the quality of this item.
Top Predictions:
- CG: 99.92%
- OR: 0.08%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step

Review: Decent product, but could use some improvements.
Top Predictions:
- OR: 99.98%
- CG: 0.02%
